<a href="https://colab.research.google.com/github/giselalsouto/LC-bluesky-sentiment/blob/b2/analizar_sentimentos_bsky_v0_3_r0002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Seleção de toots e análise de sentimentos em Google Colab

A) Seleção de posts:

    Configura bluesky-scrape para selecionar posts sobre uma entidade (indivíduo, empresa, produto, localidade, etc) que consideres relevante.

    Guarda os resultados num ficheiro e aplica o módulo de sentimentos de Linguakit.

    Analisa uma amostra dos posts, identifica os erros do sistema (mal classificados) e procura uma explicação para alguns dos casos errados.

B) Melhora o dicionário polarizado:

    Abre um dos dicionários do módulo de análise de sentimentos de Linguakit (gl, pt, en, es) e procura palavras polarizadas que não estão mas deveriam estar.

    Adiciona essas palavras ao dicionário.

    Analisa frases que contenham essas palavras.

C) Elabora o teu próprio dicionario polarizado numa das 4 línguas de Linguakit

    Cria um novo ficheiro com a lista de palavras polarizadas que consideres essenciais.

    Sube-o a Google Colab e coloca-o no lugar ajeitado com o mesmo nome que o dicionário antigo.

    Prepara uma lista de frases de teste e prova o analisador com o teu novo dicionário.

D) Entrega dos documentos para serem avaliados:

    Documento de uma ou duas folhas em PDF que contenha a ligação ao teu Google Colab e uma breve explicação do que foi feito nos pontos A, B e C: erros analisados, que novas palavras fórom adicionadas ao dicionário existente, frases utilizadas para testar o sistema com o dicionário modificado (ponto B) e com o novo dicionário (ponto C), etc.


A) Seleção de posts:

    Configura bluesky-scrape para selecionar posts sobre uma entidade (indivíduo, empresa, produto, localidade, etc) que consideres relevante.

    Guarda os resultados num ficheiro e aplica o módulo de sentimentos de Linguakit.

    Analisa uma amostra dos posts, identifica os erros do sistema (mal classificados) e procura uma explicação para alguns dos casos errados.
  

# A.1.  Descarga e instalación de bluesky-scrape.

## A.1.1. Comprobación de dependencias

Antes de descargar e instalar o software que imos empregar para facer a descarga do contido, imos ver se temos os paquetes requiridos para iso.

Neste caso, serían os paquetes `json` (para poder empregar arquivos en formato json) e `requests`que é un paquete para fecer solicitudes HTTP para recuperar información desde a Web.


In [ ]:
import json, requests
# Tamén imos importar a biblioteca os para poder navegar entre ficheiros e usar \
# os propios ficheiros.
import os


Ao non dar erros, esto indica que os paquetes xa se atopan no sistema.

## A.1.2. Instalación de biblioteca *bluesky-scrape*

Unha vez comprobadas as dependencias, e ao non ter que instalar ningunha, imos instalar, *bluesky-scrape*.

Para esta instalación, primeiro temos que descargar o repositorio, por medio de git.

In [ ]:
existe = os.path.exists('/content/bluesky-scrape/')
if ( not existe) :
  !git clone https://github.com/gamallo/bluesky-scrape.git

Cloning into 'bluesky-scrape'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 8), reused 26 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 18.57 KiB | 3.71 MiB/s, done.
Resolving deltas: 100% (8/8), done.


Agora, miramos onde descargou o git o contido mediante `ls`-

In [ ]:
!ls -lF

total 8
drwxr-xr-x 3 root root 4096 May 18 16:12 bluesky-scrape/
drwxr-xr-x 1 root root 4096 May 14 13:38 sample_data/


Como vemos, o contido está nunha carpeta chamada *bluesky-scrape*.

Os parámetros empregados no `ls` son `-l` (visualización en formato longo con variada información) e)`-F`(marcar con `/` ao final os directorios).


Agora imos a ver de por o contrasinal sen que se vexa

In [ ]:
##ISTO É SÓ PARA OCULTAR O MEU PASSWORD
from google.colab import userdata
bs_passw = userdata.get('bsky_pass')

In [ ]:
import os
os.chdir('/content/bluesky-scrape/')
if (not os.path.exists('/content/bluesky-scrape/results/')):
  os.mkdir('/content/bluesky-scrape/results/')
os.listdir()

['results',
 'old_bluesky-scraper.py',
 'bluesky-scraper.py',
 '.git',
 'LICENSE',
 'README.md']

In [ ]:
import shutil


In [ ]:
import requests
import json
import sys
from datetime import datetime, timedelta

username = "giselals.bsky.social" ## Coloca aqui o teu usuário
password = bs_passw  # Coloca aqui o teu password
keyword = "israel"

limit = 25 ##número máximo de posts
since = "2025-04-01" #data de início
until = "2025-05-16" #data de fim
lang = "es" #língua do post: en, es, gl, pt, fr, ...

BASE_URL = "https://bsky.social/xrpc/app.bsky.feed.searchPosts"



In [ ]:
def authenticate(username, password):
    """
    Autenticar na Bluesky e obter um token de acesso.
    """
    auth_url = "https://bsky.social/xrpc/com.atproto.server.createSession"
    data = {
        "identifier": username,
        "password": password
    }
    response = requests.post(auth_url, json=data)
    response.raise_for_status()
    return response.json().get("accessJwt")

In [ ]:
def search_posts(keyword, access_token, start_date=since, end_date=until, limit=limit, lang=None):
    """
    Pesquisa por mensagens que contenham a palavra-chave especificada.
    """
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "q": keyword,
        "limit": limit
    }
    if start_date:
        params["since"] = f"{start_date}T00:00:00Z"
    if end_date:
        params["until"] = f"{end_date}T23:59:59Z"
    if lang:
        params["lang"] = lang

    response = requests.get(BASE_URL, headers=headers, params=params)
    response.raise_for_status()
    return response.json().get("posts", [])



In [ ]:
def save_to_file(keyword, posts):
    """
    Guardar os posts num ficheiro json.
    """
    filename = f"{keyword.replace(' ', '_')}_results.json"
    with open(filename, "w", encoding="utf-8") as file:
        json.dump(posts, file, indent=4, ensure_ascii=False)
    print(f"Guardar resultados de '{keyword}' em {filename}")



In [ ]:
def save_text_only(keyword, posts):
    """
    Guardar só o texto dos posts num ficheiro .txt
    """
    filename = f"{keyword.replace(' ', '_')}_results.txt"
    with open(filename, "w", encoding="utf-8") as file:
        for post in posts:
            text = post.get("record", {}).get("text", "")
            if text:
                file.write(text.replace("\n", " ").strip() + "\n")
    print(f"Guardar só texto sobre '{keyword}' em {filename}")

In [ ]:
def scrape():

    # Autenticar-se
    try:
        access_token = authenticate(username, password)
    except requests.exceptions.RequestException as e:
        print(f"Authentication error: {e}")
        sys.exit(1)

    # Procurar posts
    try:
        posts = search_posts(keyword, access_token, start_date=since, end_date=until, limit=limit, lang=lang)
    except requests.exceptions.RequestException as e:
        print(f"Search error: {e}")
        sys.exit(1)

    for post in posts:
        text = post.get("record", {}).get("text", "No content")
        author = post.get("author", {}).get("displayName", "Unknown Author")
        created_at = post.get("record", {}).get("createdAt", "Unknown Date")
        print(f"Author: {author}\nCreated At: {created_at}\nContent: {text}\n{'-' * 40}")

    # Guardar posts
    save_to_file(keyword, posts)
    save_text_only(keyword, posts)

In [ ]:
import re
def replace_punctuation ( text, replace ):
  return re.sub('[%s]' % re.escape(string.punctuation), replace, text)

In [ ]:
scrape()

Author: Luicántropo
Created At: 2025-05-16T23:58:05.634Z
Content: Tengo ya depresión post-eurovisión, y eso que aún no ha acabado. Estoy tan acostumbrado a venirme abajo después de acontecimientos muy esperados, que ya mi cerebro lo anticipa.

Tampoco ayuda la sensación enorme de culpa que tengo por ver el festival pese a que apoye y valide a Israel.
----------------------------------------
Author: Rubén Darío
Created At: 2025-05-16T23:56:32.014Z
Content: Israel OG
🖼️
Soy enemigo de los lugares llenos de gente, de la fanfarronería de dinero y de linaje, de la pretensión, de la gente consumidora, de las relaciones fugaces y de las reuniones llenas de hipocresía y mentiras.
 ⁣
   ☯️     Fiódor Dostoievski
----------------------------------------
Author: Manuel Ibáñez
Created At: 2025-05-16T23:56:31.441Z
Content: INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo.
BIEN!!! 👍👍💪🏼🇵🇸
--------------------------

In [ ]:
import shutil
#xeramos os nomes de arquivo
txt = '.txt'
json = '.json'
results = '_results'
dirres = './results/'
res_txt = keyword + results + txt
res_json = keyword + results + json
path_res_txt = dirres + res_txt
path_res_json = dirres + res_json
##Se non existe o ficheiro de resultados no directorio results
#if (not os.path.exists (path_res_txt)):
#Entón movemos o arquivo de resultados ao directorio results
shutil.move(res_txt , path_res_txt)
##Se non existe o ficheiro de resultados no directorio results
#if (not os.path.exists (path_res_json)):
#Entón movemos o arquivo de resultados ao directorio results
shutil.move(res_json , path_res_json)
!ls ./results
!ls

israel_results.json  israel_results.txt
bluesky-scraper.py  LICENSE  old_bluesky-scraper.py  README.md	results


## A.2. Analizar o texto completo

# Imos a

Aquí :

Primeiro creamos as varaibles precisas (`texto_txt`e `texto_json`)

Logo,buscamos os elementos da ruta de cada ficheiro:

- Primeiro, pomos o directorio do ficheiro
- Logo, montamos o nome do ficheiro por medio dos seus elementos:
  - o elemento keyword (que é o termo que buscamos en bluesky)
  - o elemento results ( que engade `_results` ao nome)
  - a extensión ( que engade `.txt` ou `.json`, segundo o tipo de texto )

Todo isto facémolo para pasar o contido dos arquivos tanto de texto, como JSON  a cadansúa variable.

In [ ]:
#Créamo-la variable de texto texto_txt, para conter todos os posts Bluesky
texto_txt = ''
#Creamos vairable JSON texto_json para ter os posts en formato JSON
texto_json = ''
#parte de directorio da
dir_file_txt  = ('/content/bluesky-scrape/results/')
dir_file_json = ('/content/bluesky-scrape/results/')
results = '_results'
print('results: ', results)
file_txt_path = dir_file_txt + keyword + results + txt
print('Ruta do ficheiro de texto: ',file_txt_path)
file_json_path = dir_file_txt + keyword + results + json
print('Ruta do ficheiro JSON: ', file_json_path)


results:  _results
Ruta do ficheiro de texto:  /content/bluesky-scrape/results/israel_results.txt
Ruta do ficheiro JSON:  /content/bluesky-scrape/results/israel_results.json


In [ ]:

# Se o ficheiro existe
if (os.path.exists(file_txt_path)):
  #Abrímo-lo para lectura
  with (open(file_txt_path,'r')) as file_txt:
    #E creámo-la variable texto
    texto_txt = file_txt.read()

if (os.path.exists(file_json_path)):
  with (open(file_json_path,'r')) as file_json:
    texto_json = file_json.read()

print ('Texto ( ', limit, ' posts de Bluesky) : ', texto_txt)
print ('JSON ( ', limit, ' posts de Bluesky) : ',texto_json)


Texto (  25  posts de Bluesky) :  Tengo ya depresión post-eurovisión, y eso que aún no ha acabado. Estoy tan acostumbrado a venirme abajo después de acontecimientos muy esperados, que ya mi cerebro lo anticipa.  Tampoco ayuda la sensación enorme de culpa que tengo por ver el festival pese a que apoye y valide a Israel.
Israel OG 🖼️ Soy enemigo de los lugares llenos de gente, de la fanfarronería de dinero y de linaje, de la pretensión, de la gente consumidora, de las relaciones fugaces y de las reuniones llenas de hipocresía y mentiras.  ⁣    ☯️     Fiódor Dostoievski
INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo. BIEN!!! 👍👍💪🏼🇵🇸
Si después de esto no betan a Israel ojalá y nos retiremos. Que el Benidorm Fest continúe pero sin ir a Eurovisión y fin, tendremos nuestro propio festival y sin genocidas.
EEUU. Lo detuvieron porque está contra Israel. El genocidio en gaza.
España acaba de reconocer que si

Aquí facemos unha lista lines coas liñas (cada unha é un *post* de Bluesky).
Isto facémolo para prever as probables necesidades futuras de análise de cada post (cada un está nunha liña), así como de frases ou palabras.

In [ ]:
lines = []
with (open(file_txt_path,'r')) as file_txt:
  for i in range(limit):
    line = file_txt.readline()
    lines.append(line)
    #lines[i] = line
    print('line', " ", i, " :", line)

print('\n \n')
print(' ============= LIÑAS =============== \n \n \n')
print (lines)


line   0  : Tengo ya depresión post-eurovisión, y eso que aún no ha acabado. Estoy tan acostumbrado a venirme abajo después de acontecimientos muy esperados, que ya mi cerebro lo anticipa.  Tampoco ayuda la sensación enorme de culpa que tengo por ver el festival pese a que apoye y valide a Israel.

line   1  : Israel OG 🖼️ Soy enemigo de los lugares llenos de gente, de la fanfarronería de dinero y de linaje, de la pretensión, de la gente consumidora, de las relaciones fugaces y de las reuniones llenas de hipocresía y mentiras.  ⁣    ☯️     Fiódor Dostoievski

line   2  : INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo. BIEN!!! 👍👍💪🏼🇵🇸

line   3  : Si después de esto no betan a Israel ojalá y nos retiremos. Que el Benidorm Fest continúe pero sin ir a Eurovisión y fin, tendremos nuestro propio festival y sin genocidas.

line   4  : EEUU. Lo detuvieron porque está contra Israel. El genocidio en gaza.

l

In [ ]:
import string
#primera = []
#primera = lines[0].split()
#print (primera)

p = 0
f = 0
frases = []
palabras = []
for line in lines:
  frases.append(line.split('.'))
  palabras.append(line.split(' '))
print (frases)
print (palabras)

for fi in range(limit):
  f += len(frases[fi])
print('Número total de frases: ', f)

for pi in range(fi):
  p += len(palabras[pi])

print('Número total de palabras: ', p)

[['Tengo ya depresión post-eurovisión, y eso que aún no ha acabado', ' Estoy tan acostumbrado a venirme abajo después de acontecimientos muy esperados, que ya mi cerebro lo anticipa', '  Tampoco ayuda la sensación enorme de culpa que tengo por ver el festival pese a que apoye y valide a Israel', '\n'], ['Israel OG 🖼️ Soy enemigo de los lugares llenos de gente, de la fanfarronería de dinero y de linaje, de la pretensión, de la gente consumidora, de las relaciones fugaces y de las reuniones llenas de hipocresía y mentiras', '  \u2063    ☯️     Fiódor Dostoievski\n'], ['INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo', ' BIEN!!! 👍👍💪🏼🇵🇸\n'], ['Si después de esto no betan a Israel ojalá y nos retiremos', ' Que el Benidorm Fest continúe pero sin ir a Eurovisión y fin, tendremos nuestro propio festival y sin genocidas', '\n'], ['EEUU', ' Lo detuvieron porque está contra Israel', ' El genocidio en gaza', '\

In [ ]:
# Instalar linguakit
os.chdir('/content')
!pwd
!git clone https://github.com/citiususc/Linguakit.git
os.listdir()

/content
Cloning into 'Linguakit'...
remote: Enumerating objects: 2683, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 2683 (delta 33), reused 71 (delta 25), pack-reused 2589 (from 1)
Receiving objects: 100% (2683/2683), 330.19 MiB | 19.83 MiB/s, done.
Resolving deltas: 100% (1656/1656), done.
Updating files: 100% (328/328), done.


['.config', 'bluesky-scrape', 'Linguakit', 'sample_data']

In [ ]:
#[OPCIONAL]
!cpan LWP::UserAgent
!cpan PerlIO::gzip
!cpan HTTP::Request::Common

Streaming output truncated to the last 5000 lines.
#     ------------------- ---- ----
#     ExtUtils::MakeMaker 6.17 7.62
# 
# === Build Requires ===
# 
#     Module              Want Have
#     ------------------- ---- ----
#     ExtUtils::MakeMaker  any 7.62
# 
# === Test Requires ===
# 
#     Module              Want     Have
#     ------------------- ---- --------
#     ExtUtils::MakeMaker  any     7.62
#     File::Spec           any     3.80
#     IO::File             any     1.46
#     Test::More          0.62 1.302183
#     lib                  any     0.65
# 
# === Test Recommends ===
# 
#     Module         Want     Have
#     ---------- -------- --------
#     CPAN::Meta 2.120900 2.150010
# 
# === Runtime Requires ===
# 
#     Module        Want     Have
#     ------------ ----- --------
#     Carp           any     1.52
#     Exporter       any     5.76
#     File::Spec     any     3.80
#     File::Temp     any   0.2311
#     IO::Handle     any     1.46
#     Scalar::Util  

In [ ]:
#print (file_txt_path)

In [ ]:
!cat /content/bluesky-scrape/results/israel_results.txt |/content/Linguakit/linguakit sent en > israel.csv
!cat israel.csv

Tengo ya depresión post-eurovisión , y eso que aún no ha acabado . Estoy tan acostumbrado a venirme abajo después de acontecimientos muy esperados , que ya mi cerebro lo anticipa . Tampoco ayuda la sensación enorme de culpa que tengo por ver el festival pese a que apoye y valide a Israel . 	NEGATIVE	0.514053135757777
Israel OG 🖼️ Soy enemigo de los lugares llenos de gente , de la fanfarronería de dinero y de linaje , de la pretensión , de la gente consumidora , de las relaciones fugaces y de las reuniones llenas de hipocresía y mentiras . ⁣ ☯️ Fiódor_Dostoievski   	NONE	1
INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo . BIEN ! ! ! 👍👍💪🏼🇵🇸   	POSITIVE	0.607016212681225
Si después de esto no betan a Israel ojalá y nos retiremos . Que el Benidorm_Fest continúe pero sin ir a Eurovisión y fin , tendremos nuestro propio festival y sin genocidas . 	NEGATIVE	0.999999999859967
EEUU . Lo detuvieron porque est

### A.3) Análise de Mostras

Aquí imos analizar algunhas mostras.

Para comezar, imos sinalar que nalgunha das execucións do guión, colóusenos algún microartigo que non falaba do que buscabamos (Israel como país) senón mencionaba a unha persoa de tal nome.

Por isto, en caso de darse isto, non empregaremos esa mostra concreta, agás que sexa de grande interese para o noso caso (por exemplo, que vexamos que o valor de negatividade e positividade se desvía moito do que vexamos lóxico).



Primeiro, tomaremos as mostras a analizar.

In [ ]:
mostras = []
for i_mostras in range(2,8):
  mostras.append(lines[i_mostras])
print(mostras)


['INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo. BIEN!!! 👍👍💪🏼🇵🇸\n', 'Si después de esto no betan a Israel ojalá y nos retiremos. Que el Benidorm Fest continúe pero sin ir a Eurovisión y fin, tendremos nuestro propio festival y sin genocidas.\n', 'EEUU. Lo detuvieron porque está contra Israel. El genocidio en gaza.\n', 'España acaba de reconocer que si sigue haciendo negocios con Israel es cómplice de un genocidio.\n', 'There probably will be. Ime, every person I know whose ancestors escaped pogroms tend to believe Israel as a country should exist.\n', 'Porque aceptan a Israel? Si no tiene nada que ver con Europa. O acaso permiten participar a países no europeos? Pregunto por mi ignorancia\n']


In [ ]:
print(mostras[1])

Si después de esto no betan a Israel ojalá y nos retiremos. Que el Benidorm Fest continúe pero sin ir a Eurovisión y fin, tendremos nuestro propio festival y sin genocidas.



Logo, pasaremos a un arquivo as mostras.

In [ ]:
dir_file_mostras = '/content/resultados_linguakit_mostras/'
if (not os.path.exists(dir_file_mostras)):
  os.mkdir(dir_file_mostras)
results='_Lk_results';
print(len(mostras))
file_mostras_path = dir_file_mostras + keyword + txt
print ('file_mostras_path: ',file_mostras_path)
with (open(file_mostras_path,'w')) as file_mostras:
  for m in range(len(mostras)):
    m_file_line = mostras[m] + '\n'
    file_mostras.write(m_file_line)

6
file_mostras_path:  /content/resultados_linguakit_mostras/israel.txt


Ahora que tenemos la información en el archivo que tiene por ruta `/content/resultados_linguakit_mostras/israel_LK_results.txt`, vamos a procesarlo con *Linguakit* .

In [ ]:
!cat /content/resultados_linguakit_mostras/israel.txt |/content/Linguakit/linguakit sent en > /content/resultados_linguakit_mostras/israel_linguakit.csv
!cat /content/resultados_linguakit_mostras/israel_linguakit.csv

INFOLIBRE no publicará información sobre el festival de EUROVISIÓN en solidaridad con Gaza y por la participación de Israel en el mismo . BIEN ! ! ! 👍👍💪🏼🇵🇸   	POSITIVE	0.607016212681225
Si después de esto no betan a Israel ojalá y nos retiremos . Que el Benidorm_Fest continúe pero sin ir a Eurovisión y fin , tendremos nuestro propio festival y sin genocidas . 	NEGATIVE	0.999999999859967
EEUU . Lo detuvieron porque está contra Israel . El genocidio en gaza . 	NONE	1
España acaba de reconocer que si sigue haciendo negocios con Israel es cómplice de un genocidio . 	NEGATIVE	0.990260085055397
There probably will be . Ime , every person I know whose ancestors escaped pogroms tend to believe Israel as a country should exist . 	NEGATIVE	0.999997578434696
Porque aceptan a Israel ? Si no tiene nada que ver con Europa . O acaso permiten participar a países no europeos ? Pregunto por mi ignorancia   	NEGATIVE	0.999933392481952
TOTAL	NEGATIVE	0.563862473858464


##B)